<a href="https://colab.research.google.com/github/Chiaradisanto/Segmentation/blob/main/FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  **Save DICOM FILES as .png images**



Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Importing libraries


In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd 
import os
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from  scipy import ndimage



Installing and importing module for reading and writing DICOM files

In [ ]:
!pip install pydicom
import pydicom
from pydicom import dcmread


**get_names** function takes as input the path of the selected subject containing the DICOM files 



In [ ]:
def get_names(path):
    names = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            _, ext = os.path.splitext(filename)
            if ext in ['.dcm']:
                names.append(filename)
    return names

DICOM files are ordered with sort function. 

In [ ]:
names = sorted(get_names(path)) # path of the .dicom files

A list containig all the DICOM files is created. 
This operation loads all DICOM images from the folder into a list for manipulation.

In [ ]:
files = []
for x in names:
        files.append(pydicom.dcmread(path+'/'+x))
len(files)


Make sure that all the files contains SliceLocation in it. Otherwhise skip them



In [ ]:
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

Slices are sorted according to SliceLocation

In [ ]:
slices = sorted(slices, key=lambda s: s.SliceLocation, reverse=True)  #reverse is True to sort in descending order


The voxel values in the images are raw. 

**dicom_HU** converts raw values into Houndsfeld units.
This function takes as input all the slices of considered subject.
The transformation is linear. 

Both the rescale intercept and rescale slope are stored in the DICOM header at the time of image acquisition.
The final value is rescaled to HU.
Windowing is then applied in order to adjust the grayscale level of the images.




In [ ]:
def dicom_HU(scan):
    image = np.stack([s.pixel_array*s.RescaleSlope+s.RescaleIntercept for s in scan],axis=2).astype(np.int16) #

    img_min = 120 - 120 // 2 #minimum HU level
    img_max = 120 + 120 // 2 #maximum HU level
    window_image = image.copy()

    window_image[window_image < img_min] = img_min #set img_min for all HU levels less than minimum HU level
    window_image[window_image > img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    plt.figure(figsize=(20, 10))
  
    plt.style.use('grayscale') 
    plt.imshow(window_image[:,:,0], cmap='gray')
    plt.axis('off')
    return  np.array(window_image)

Save as images in png format in a specific folder. A folder for each patient was created.

In [ ]:
num_slices=len(files)
for i in range(1,num_slices):
    scan=dicom_HU(slices[i-1:i])
    plt.axis('off')
    plt.savefig(f"{images_path}"+str(i)+".png", bbox_inches='tight',pad_inches = 0,dpi=300, quality=95) #dpi represents the resolution in dots per inch.
                                                                                                        # A high value results in a high resolution image.
                                                                                                    
                                                                                                        #quality represents the image quality
                                                                                                        #on a scale from 1 (worst) to 95 (best)
    plt.show()

These operations must be repeated for all the subjects changing the paths.

### Save **ROI.nrrd** files as png. masks

Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Installing SimpleITK to read .nrrd files 

In [ ]:
!pip install SimpleITK


Importing libraries


In [3]:
import glob
import os
import SimpleITK as sitk
import nibabel as nib
from matplotlib import pyplot as plt
import numpy as np
import numpy as np


In [4]:
ROI_path= glob.glob('/gdrive/MyDrive/ROI_out.nrrd') # Path of the .nrrd file


Read the .nrrd files

In [ ]:
mask_sitk  = sitk.ReadImage(ROI_path[0]) #read the masks
sitk_shape=mask_sitk.GetSize() #get the masks size
print(sitk_shape) # the first 2 dimensions are height and width , the last dimension represents the number of masks in the folder

Save as masks in .png format in a  specific folder.
A folder for each subject was created.

In [ ]:
num_slices=sitk_shape[2]
for i in range(1,num_slices):
    img=plt.imshow(sitk.GetArrayViewFromImage(mask_sitk)[i], cmap='gray')    
    plt.axis('off')
    plt.savefig(f"{images_path}"+str(i)+".png", bbox_inches='tight',pad_inches = 0,dpi=300, quality=95) #dpi represents the resolution in dots per inch.
                                                                                                        # A high value results in a high resolution image.
                                                                                                    
                                                                                                        #quality represents the image quality
                                                                                                        #on a scale from 1 (worst) to 95 (best)
    plt.show()


These operations must be repeated for all the subjects changing the paths.

21 folders were created, one for each subject. At the end of these steps, each folder contains two subfolders "images" and "masks" with the associated images and masks for each subject.
Subjects's folders are splitted in TRAINING and VALIDATION folders.
18 subjects are in TRAINING folder while 3 subjects are in VALIDATION one.
7-fold cross validation was performed manually.

### Final folders creation

Once the images and masks of each subject have been saved ,in the next part of the code four folders have been created: 'train_images' and 'train_masks' in which are all the images and the associated masks of the training set, and 'validation_images' and 'validation_masks' which contains the images and the associated masks of the validation set. This operation was necessary to perform Data Augumentation in a correct way, described in the later steps of the code.
The images and the associated masks are charaterized by the same name.

Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Importing libraries

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
from glob import glob
from tqdm import tqdm
from natsort import natsorted
from google.colab.patches import cv2_imshow


In [ ]:
TRAIN_path='/gdrive/MyDrive/TRAIN' #training path
VALIDATION_path='/gdrive/MyDrive/VALIDATION'# validation path

Load the images and masks of training set

In [ ]:
def load_train_data(train_path):

    train_x = (glob(f"{TRAIN_path}/*/images/*.png"))
    train_y = (glob(f"{TRAIN_path}/*/masks/*.png"))
 
    return train_x,train_y

In [ ]:
(train_x,train_y) = load_train_data(TRAIN_path)


Images and Masks of the training set are sorted naturally with natsorted function 

In [ ]:
train_x=natsorted(train_x)
train_y=natsorted(train_y)
print(len(train_x))   #train_x and train_y must have the same length , since they must contains the same number of files.
print(len(train_y))

Load the images and masks of validation set


In [ ]:
def load_val_data(val_path):
    val_x= (glob(f"{VALIDATION_path}/*/images/*.png"))
    val_y= (glob(f"{VALIDATION_path}/*/masks/*.png"))
    return val_x,val_y

In [ ]:
(val_x,val_y) = load_val_data(VALIDATION_path)


Images and Masks of the validation set are sorted naturally with natsorted function.


In [ ]:
val_x=natsorted(val_x)
val_y=natsorted(val_y)
print(len(val_x))
print(len(val_y))

Creating the 'train_images' and 'train_masks' folders

In [ ]:
image_path_train='/gdrive/MyDrive/train_images/images/' # final images path
mask_path_train='/gdrive/MyDrive/train/train_masks/masks/' #final masks path

In [ ]:
H = 256 #height 
W = 256 #width
for idx, (x, y) in tqdm(enumerate(zip(train_x, train_y)), total=len(train_x)):
        """ Extracting the folder name and image name for each subject """
        dir_name = x.split("/")[-3]
        name = dir_name + "_" + x.split("/")[-1].split(".")[0] #extract the name of every subject's folder 
        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_GRAYSCALE) #read the image
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE) #read the mask
        X = [x]
        Y = [y]
        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H),interpolation = cv2.INTER_CUBIC)   #images are resized from 512x512 to 256x256 through cubic interpolation
            m = cv2.resize(m, (W, H),interpolation = cv2.INTER_CUBIC)   #masks are resized from 512x512 to 256x256 through cubic interpolation

            tmp_image_name = f"{name}.png"  #image and mask are saved with the same name 
            tmp_mask_name  = f"{name}.png" 

            image_path = os.path.join(image_path_train, tmp_image_name)
            mask_path  = os.path.join(mask_path_train, tmp_mask_name)
            print(tmp_image_name)

            cv2.imwrite(image_path,i, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save images in the folder
            cv2.imwrite(mask_path, m, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save masks in the folder
                                                                            #with cv2.IMWRITE_PNG_COMPRESSION parameter the compression of the png image
                                                                            #can be controlled. The value 0 pruduces the lowest compression
            idx += 1

Creating the 'validation_images' and 'validation_masks' folders

In [ ]:
image_path_validation='/gdrive/MyDrive/validation_images/images/' # final images path
mask_path_validation='/gdrive/MyDrive/train/validation_masks/masks/' #final masks path

In [ ]:
H = 256 #height 
W = 256 #width
for idx, (x, y) in tqdm(enumerate(zip(val_x, val_y)), total=len(val_x)):
        """ Extracting the folder name and image name for each subject """
        dir_name = x.split("/")[-3]
        name = dir_name + "_" + x.split("/")[-1].split(".")[0] #extract the name of every subject's folder 
        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_GRAYSCALE) #read the image
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE) #read the mask
        X = [x]
        Y = [y]
        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H),interpolation = cv2.INTER_CUBIC)   #images are resized from 512x512 to 256x256 through cubic interpolation
            m = cv2.resize(m, (W, H),interpolation = cv2.INTER_CUBIC)   #masks are resized from 512x512 to 256x256 through cubic interpolation

            tmp_image_name = f"{name}.png"  #image and mask are saved with the same name 
            tmp_mask_name  = f"{name}.png" 

            image_path = os.path.join(image_path_validation, tmp_image_name)
            mask_path  = os.path.join(mask_path_validation, tmp_mask_name)
            print(tmp_image_name)

            cv2.imwrite(image_path,i, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save images in the folder
            cv2.imwrite(mask_path, m, [int(cv2.IMWRITE_PNG_COMPRESSION),0]) #save masks in the folder
                                                                            #with cv2.IMWRITE_PNG_COMPRESSION parameter the compression of the png image
                                                                            #can be controlled. The value 0 pruduces the lowest compression
            idx += 1

###Data Augmentation

Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Importing Libraries


In [ ]:
from matplotlib import pyplot as plt
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
import cv2
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
#from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, add, concatenate,UpSampling2D,ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train_img_path = "/gdrive/MyDrive/train_images" # training images path
train_mask_path = "/gdrive/MyDrive/train_masks" # training masks path

val_img_path = "/gdrive/MyDrive/validation_images" # validation images path
val_mask_path = "/gdrive/MyDrive/validation_masks" # validation masks path

Data Augmentation is applied simultaneously to trainining images and masks

In [ ]:
img_data_gen_args_train = dict(rescale=1./255,   #rescale the images
                     rotation_range=90,          #random rotation 
                     brightness_range=[0.3,1.5], # random change of brightness
                     width_shift_range=0.3,      # random width shifting
                     height_shift_range=0.3,     #random height shifting
                     shear_range=0.5,            #shear transformation
                     horizontal_flip=True,       #horizontal flip
                     vertical_flip=True,         # vertical flip
                     fill_mode='reflect')        # fill mode

mask_data_gen_args_train = dict(
                     rotation_range=90,          #random rotation 
                     brightness_range=[0.3,1.5], # random change of brightness
                     width_shift_range=0.3,      # random width shifting
                     height_shift_range=0.3,     #random height shifting
                     shear_range=0.5,            #shear transformation
                     horizontal_flip=True,       #horizontal flip
                     vertical_flip=True,         # vertical flip
                     fill_mode='reflect',       # fill mode

                     preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype) #Binarize the masks.
                     
                     )  

image_data_generator_train = ImageDataGenerator(**img_data_gen_args_train)
mask_data_generator_train = ImageDataGenerator(**mask_data_gen_args_train)

Data Augmentation is not applied to validation images and masks.
ImageDataGenerator is used to rescale the images and binarize the masks only.

In [ ]:
img_data_gen_args_val = dict(rescale=1./255) #rescale the images

mask_data_gen_args_val = dict(
                     preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype)
                     
                     ) #Binarize the masks. 

image_data_generator_val = ImageDataGenerator(**img_data_gen_args_val)
mask_data_generator_val = ImageDataGenerator(**mask_data_gen_args_val)

ImageDataGenerator produces 2 generators,one for training and one for validation with the images and associated masks.
These generators will be the input of the model.

In [ ]:
seed=42
batch_size=16
image_generator = image_data_generator_train.flow_from_directory(train_img_path, #path
                                                           seed=seed, # must be the same to ensure that images and masks are edited in the same way
                                                           batch_size=batch_size,
                                                           color_mode = 'grayscale', #Read images in grayscale
                                                           target_size=(256,256), #specify the height and the width
                                                           class_mode=None)  #Very important to set this otherwise it returns multiple numpy arrays 
                                                                            

mask_generator = mask_data_generator_train.flow_from_directory(train_mask_path, 
                                                         seed=seed, 
                                                         batch_size=batch_size,
                                                         color_mode = 'grayscale',
                                                         target_size=(256,256)  , 
                                                         class_mode=None)


valid_img_generator = image_data_generator_val.flow_from_directory(val_img_path, 
                                                               seed=seed, 
                                                               batch_size=batch_size, 
                                                               color_mode = 'grayscale', 
                                                               target_size=(256,256),
                                                               class_mode=None) 


valid_mask_generator = mask_data_generator_val.flow_from_directory(val_mask_path, 
                                                               seed=seed, 
                                                               batch_size=batch_size, 
                                                               target_size=(256,256),
                                                               
                                                               color_mode = 'grayscale',  
                                                               class_mode=None)  


train_generator = zip(image_generator, mask_generator)
val_generator = zip(valid_img_generator, valid_mask_generator)

Show some example of augmented images and check if they are associated in correct way to the masks

In [ ]:

x, y = train_generator.__next__()

for i in range(0,8):
    image = x[i,:,:,0]
    mask= y[i,:,:,0]
    plt.subplot(1,2,1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    plt.show()

Check if images and masks are rescaled. 
Values must be between 0 and 1 for images and 0 or 1 for masks

In [ ]:
print(x.max())
print(y.max())

### Model

Define the input shape of the model

In [ ]:
IMG_HEIGHT = x.shape[1]
IMG_WIDTH  = x.shape[2]
IMG_CHANNELS = x.shape[3]
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
print(input_shape)

Unet-based Architecture

In [ ]:
inputs = tf.keras.layers.Input(input_shape)
#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1= tf.keras.layers.BatchNormalization()(c1)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
c1= tf.keras.layers.BatchNormalization()(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2= tf.keras.layers.BatchNormalization()(c2)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
c2= tf.keras.layers.BatchNormalization()(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3= tf.keras.layers.BatchNormalization()(c3)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
c3= tf.keras.layers.BatchNormalization()(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4= tf.keras.layers.BatchNormalization()(c4)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
c4= tf.keras.layers.BatchNormalization()(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5= tf.keras.layers.BatchNormalization()(c5)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
c5= tf.keras.layers.BatchNormalization()(c5)
#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.summary()

Define the steps per epoch for training and validation required to train the model

In [ ]:
num_train_imgs = len(os.listdir("/gdrive/MyDrive/train_images/images")) #images path
num_val_images = len(os.listdir("/gdrive/MyDrive/val_images/images"))   #masks path

steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size


Define the model metrcis and losses.
Dice Loss, IoU Loss, Tversky Loss, Focal Loss have been tested

In [ ]:
from keras import backend as K


def dice_coefficient(y_true, y_pred, smooth=0.0001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    intersection = K.sum(y_true_f * y_pred_f)

    return ((2. * intersection + smooth) / (K.sum(y_true_f) +
            K.sum(y_pred_f) + smooth))


def dice_coefficient_loss(y_true, y_pred):
    return 1.0-dice_coefficient(y_true, y_pred)


def iou_loss(y_true, y_pred):
    return 1-iou(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred))
    sum_ = K.sum(K.square(y_true)) + K.sum(K.square(y_pred))
    jac = (intersection) / (sum_ - intersection)
    return jac

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)


Define Learning Rate and Optimizer

In [ ]:
LR = 5e-5
optim = tf.keras.optimizers.Adam(LR) #Adaptive Moment Estimation Optimizer

Final Metrics used

In [ ]:
metrics = [iou, dice_coefficient, 'binary_accuracy']


Compile the model

In [ ]:
model.compile(optimizer=optim, loss=dice_coefficient_loss, metrics=metrics)


Add callback to save the best model 

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/gdrive/MyDrive/chk/',
    save_weights_only=True,
    monitor='val_dice_coefficient',
    mode='max',
    save_best_only=True)

Train the model

In [ ]:
history=model.fit(train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=50,
          verbose=1,
          callbacks=model_checkpoint_callback,
          validation_data=val_generator,
          validation_steps=val_steps_per_epoch)

Save the model in .h5 format

In [ ]:
model_path='/gdrive/MyDrive/model.h5'

In [ ]:
model.save(model_path)
